# Benchmarks of Strategies for Selecting Outdated Items
This notebook contains the benchmarks related to the selection strategies for context data, which we report in our paper.
Context data are selected from the processed data and included in the next progressive computation step, such that its result approximates that of a _non-progressive_ computation over the processed data.

## Benchmark Configuration

We use the following configuration in our benchmarks:
### Test cases 
- full computation over the entire dataset (upper baseline)
- progressive computation without optimization (lower baseline)
- full computation of processed data
- progressive computation using optimization strategies

### Dataset
- NYC taxis dataset (10 Million items), stored in a compressed CSV file, loaded with DuckDB 

### Variables
- dependent variables: runtime, prediction error
- independent variables: 

## Benchmarks

In [4]:
import os
from sys import path
cwd = os.getcwd()
path.append(f"{cwd}/..")

from database import initialize_db, drop_tables
from benchmark_test_case import *
import numpy as np
import pandas as pd

from doi_component.outlierness_component import OutliernessComponent
outlierness = OutliernessComponent(["ratio", "duration"])

n_dims = 17
total_size = 99999
chunk_size = 1000
chunks = round(total_size / chunk_size)

data_path = "../data/nyc_taxis.shuffled_full.csv.gz"
column_data_path = "../data/nyc_taxis.shuffled_full.parquet"

def reset():
  drop_tables()
  initialize_db(data_path, column_data_path)

from outdated_item_selection_strategy.no_update import *
from outdated_item_selection_strategy.oldest_chunks_update import *
from outdated_item_selection_strategy.last_n_chunks_update import *
from outdated_item_selection_strategy.regular_interval_update import *
from outdated_item_selection_strategy.outdated_bin_update import *

update_strategies = [
  ("no chunk", NoUpdate(n_dims=n_dims)),
  ("oldest n chunks", OldestChunksUpdate(n_dims=n_dims, n_chunks=3, max_age=10)),
  ("last n chunks", LastNChunksUpdate(n_dims=n_dims, n_chunks=3)),
  ("regular intervals", RegularIntervalUpdate(n_dims=n_dims,interval=2, max_age=10)),
  ("outdated bins", OutdatedBinUpdate(n_dims=n_dims))
]

from context_item_selection_strategy.no_context import * 
from context_item_selection_strategy.chunk_based_context import *
from context_item_selection_strategy.sampling_based_context import *
from context_item_selection_strategy.clustering_based_context import *

context_strategies = [
  ("no context", NoContext(n_dims=n_dims)),
  ("chunk based", RandomChunkBasedContext(n_dims=n_dims, n_chunks=3)),
  ("sampling based", RandomSamplingBasedContext(n_dims=n_dims, n_samples=chunk_size)),
  ("clustering based", ClusteringBasedContext(n_dims=n_dims, n_clusters=chunk_size))
]

####################################################################################################
####################################################################################################

# create the path for storing the benchmark results if they do not exist
path = f"./out/{total_size}/{chunk_size}"
if not exists ("./out"):
  os.mkdir("./out")
if not exists(f"./out/{total_size}"):
  os.mkdir(f"./out/{total_size}")
if not exists(path):
  os.mkdir(path)

### Finding an appropriate `chunk size`
The number of items in each chunk dictate the computation time for each chunk in the data, in that the more items we process, the longer the DOI computation takes.
Therefore, the first consideration in our benchmarks is to find the maximum number of items, for which the computation time remains immediate.
Prior work (see Card et al., 1991) has shown this limit to be about one second.

In the cell below, we try different chunk sizes to find the maximum items we can pass to the doi function for computations under 1s.

In [4]:
from time import time
from database import get_from_data

chunk_sizes = [10, 100, 1000, 10000]
reset()
for size in chunk_sizes:
  before = time()
  data = get_from_data([f"TRUE LIMIT {size}"], as_df=True)
  data = process_chunk(data)
  outlierness.compute_doi(data)
  print(f"{size}: {time() - before}s")
  

10: 0.3262362480163574s
100: 0.2692584991455078s
1000: 0.8617279529571533s
10000: 10.674559354782104s


### Getting the ground truth: Monolithic computation
The ground truth for our strategies is a full computation over the entire dataset without any chunking.
This computation naturally takes a long time to complete, which is why the progressive scenario is so much more effective from a user perspective: we get to see the data much faster.

In the context of the `BenchmarkTestCase` class, the monolithic computation corresponds with running a progression with a single chunk. 

In [8]:
reset()
ground_truth_test_case = BenchmarkTestCase("__ground_truth__", outlierness, NoContext(n_dims), NoUpdate(n_dims), chunk_size=total_size, chunks=1)
ground_truth_test_case.run(doi_csv_path=f"{path}/doi/")
# 99,999@1000: 20m 48.4s --> 1248s total --> 1.248s per "chunk"

### Running the test cases

In [ ]:
test_cases = []
total = len(context_strategies) * len(update_strategies)
i = 0
for c_strat in context_strategies:
  for u_strat in update_strategies:
    i += 1
    print(f"({i}/{total})context: {c_strat[0]}, update: {u_strat[0]}")
    
    # check if already completed
    label = f"{c_strat[0]}-{u_strat[0]}"
    if os.path.isfile(f"{path}/doi/{label}.csv") or os.path.isfile(f"{path}/times/{label}.csv"):
      print("skipping test case because already completed.")
      continue

    reset()
    test_case = BenchmarkTestCase(label, outlierness, c_strat[1], u_strat[1], chunk_size, chunks)
    test_case.run(doi_csv_path=f"{path}/doi/", times_csv_path=f"{path}/times/")
    test_cases += [test_case]
    print(f"done: {test_case.total_time}s")

### DOI histograms per test case

In [ ]:
import altair as alt

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{path}/doi")
available_test_cases

all_doi_values_df = pd.DataFrame()

# approach2: manually compute the histogram over all groups in the data, then visualize those bins
# histogram = np.histogram(all_doi_values_df["doi"])

def get_doi_bins_df(doi_df: pd.DataFrame) -> pd.DataFrame:
  bins = np.histogram(doi_df["doi"], bins=10, range=(0, 1))[0]
  bins_df = pd.DataFrame(bins.transpose())
  return bins_df

# compute the ground truth bins 
ground_truth_df = pd.read_csv(f"{path}/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df)

all_doi_bins_df = pd.DataFrame()

# compute the bins for each combination of strategies and then compare it to the ground truth in a 
# layered histogram
for c_strat in context_strategies:
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # read the benchmark results
    df = pd.read_csv(f"{path}/doi/{test_case}")

    # compute 10 bins on the interval [0, 1] over the "doi" column
    bins_df = get_doi_bins_df(df)

    # compute difference to the ground truth
    bins_df["delta"] = bins_df[0] - ground_truth_bins[0]
    bins_df.columns = ["doi", "delta"]

    # add context info
    bins_df["bin"] = bins_df.index
    bins_df["context_strategy"] = c_strat[0]
    bins_df["update_strategy"] = u_strat[0]

    # store these bins in a df
    all_doi_bins_df = all_doi_bins_df.append(bins_df)

alt.data_transformers.disable_max_rows()
alt.data_transformers.enable("data_server")
alt.Chart(all_doi_bins_df).mark_bar().encode(
  x=alt.X("bin:N"),
  y=alt.Y("delta:Q"),
  color={"value": "red"}
).properties(
  width=100,
  height=100
).facet(
  row="context_strategy",
  column="update_strategy",
  spacing=10
)

### Time series per test case

In [ ]:
import altair as alt

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{path}/times")
available_test_cases

all_doi_values_df = pd.DataFrame()

# build one big dataframe containing all doi scores and label each based on the strategies that were
# used to generate them
for c_strat in context_strategies:
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    df = pd.read_csv(f"{path}/times/{test_case}")
    df["context_strategy"] = c_strat[0]
    df["update_strategy"] = u_strat[0]
    all_doi_values_df = all_doi_values_df.append(df)
    all_doi_values_df.reset_index(inplace=True, drop=True)

# approach1: manually compute the histogram over all groups in the data, then visualize those bins
# histogram = np.histogram(all_doi_values_df["doi"])
# approach2: use altair to do the grouping and binning.
alt.data_transformers.disable_max_rows()
alt.Chart(all_doi_values_df).mark_line().encode(
  x="chunk:Q",
  y={"field": "total_time", "type": "quantitative", "scale": {"type": "log"}, "title": "time (s)"},
  column={"field": "update_strategy"},
  color="context_strategy:N",
).properties(
  width=200,
  height=500
)

### Evaluation

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import jaccard_score, r2_score

rus = RandomUnderSampler(random_state=0)

def evaluate_test_case(test_case: np.ndarray, ground_truth: np.ndarray):
  # score = jaccard_score(test_case, ground_truth, average="weighted")
  score = r2_score(ground_truth, test_case)
  return score

ground_truth = results_full["doi"]
context_test_case = results_context["doi"]
baseline_test_case = results_chunked["doi"]


evaluate_test_case(baseline_test_case, ground_truth), evaluate_test_case(context_test_case, ground_truth)